In [1]:
import glob
import datetime
import pandas as pd
import numpy as np

In [2]:
results_folder = '_results/'
ensemble_folder = 'ensemble/'

In [3]:
result_files = glob.glob(results_folder+'*.csv')

In [4]:
import collections
c = collections.Counter()
d = collections.defaultdict(list)
for f in result_files:
    dropped = '_'.join(f.split('\\')[1].split('_')[2:-2])
    score = float(f.split('\\')[1].split('_')[-2])
    c.update([dropped])
    d[dropped].append(score)

In [5]:
for k in d:
    print(k, c[k], np.mean(d[k]))

category_name 22 0.442073682075
brand_name_category_name 11 0.443972755392
brand_name_item_description 14 0.444303521659
category_name_item_description 14 0.442679165097
brand_name 24 0.442722964327
None 13 0.439853100645
item_description 7 0.440130596506
brand_name_category_name_item_description 8 0.443861870788


In [6]:
score = {f:float(f.split('_')[-2])**-10 for f in result_files}
score_sum = np.sum(list(score.values()))

In [7]:
summit = pd.read_csv(result_files[0])
summit['price'] = 0

In [8]:
each_weights = list()
for f in result_files:
    weight = score[f] / score_sum
    each_weights.append(weight)
#     summit['price'] += (pd.read_csv(f)['price'].values) * (weight)
    summit['price'] += np.log1p((pd.read_csv(f)['price'].values)) * (weight)

weight_sum = np.sum(each_weights)
print('weight_sum', weight_sum)
assert (weight_sum-1) < 0.0001
summit['price'] = np.expm1(summit['price'])

weight_sum 1.0


In [9]:
now = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
message = 'None'
if not message:
    message = 'None'

In [10]:
summit.to_csv(ensemble_folder+'{}_{}_{}ensemble_notrescale.csv'.format(now, message, len(each_weights)), index=False)

In [11]:
# summit['price'] = summit['price'] / weight_sum
# summit.to_csv(ensemble_folder+'{}_{}_{}ensemble_rescale.csv'.format(now, message, len(each_weights)), index=False)

In [12]:
# %%time
# import os
# datafolder = '../input/'
# filenames = os.listdir(datafolder)
# data = dict()
# for i in filenames:
#     topic, *mid, filetype = i.split('.')
#     if filetype == 'tsv':
#         data[topic] = pd.read_table(datafolder + i)
#     elif filetype == 'csv':
#         data[topic] = pd.read_csv(datafolder + i)
#     else:
#         pass

# train_df = data['train']
# train_prices = np.array(list(set(train_df['price'])))
# nearest_prices = train_prices[summit['price'].apply(lambda x: np.argmin(np.abs(train_prices-x)))]
# summit['price'] = nearest_prices
# summit.to_csv(ensemble_folder+'{}_{}_{}ensemble_notrescale.csv'.format(now, message, len(each_weights)), index=False)